2 CNN: 0.98
1 CNN: 0.9615 epoch 3
1 CNN: 0.9782 epoch 50
ABC: 0.9575 batch 100, lr 0.0001, metrics criterion 'CE', epoch 3
ABC: 0.9671 batch 100, lr 0.0001, metrics criterion 'CE', epoch 6
ABC: 0.9768 epoch 50 kernel_per_pixel 8

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
torch.cuda.is_available()
torch.cuda.set_device(0)
import atd2022

In [3]:
atd_args = dotdict()
atd_args.train = dotdict()

atd_args.name = 'atd'
atd_args.train.batch_size = 30
atd_args.predict_len = 1
atd_args.history_len = 5

atd_args.train_epochs= 100
atd_args.lr = 0.001
atd_args.criterion = 'L1'
atd_args.use_gpu = True

input_channel = 1
input_height = atd_args.history_len
input_width = 5200
kernel_size = 6
knpp = 6
knpp2 = atd_args.predict_len

atd_args.layers=[
    ('specific', (input_channel, knpp, kernel_size, input_height*input_width)),
    ('specific', (knpp, kernel_size, knpp2, input_height*input_width)),
    ('linear', (knpp2*input_height, atd_args.predict_len, (1,2), 1, (1, atd_args.predict_len))),
]

In [4]:
wt_args = dotdict()
wt_args.train = dotdict()

wt_args.name = 'wiki_traffic'
wt_args.train.batch_size = 30
wt_args.predict_len = 1
wt_args.history_len = 5

wt_args.train_epochs= 10
wt_args.lr = 0.001
wt_args.criterion = 'L1'
wt_args.use_gpu = True

input_channel = 1
input_height = wt_args.history_len
input_width = 1400
kernel_size = 3
knpp = 5
knpp2 = 15

wt_args.layers=[
    ('specific', (input_channel, knpp, kernel_size, input_height*input_width)),
    ('specific', (knpp, kernel_size, knpp2, input_height*input_width)),
    ('linear', (knpp2*input_height, wt_args.predict_len, (1,2), 1, (1, wt_args.predict_len))),
]

In [5]:
lat_args = dotdict()
lat_args.train = dotdict()

lat_args.name = 'lat'
lat_args.train.batch_size = 30
lat_args.predict_len = 1
lat_args.history_len = 5

lat_args.train_epochs= 10
lat_args.lr = 0.001
lat_args.criterion = 'L1'
lat_args.use_gpu = True

input_channel = 1
input_height = lat_args.history_len
input_width = 207
kernel_size = 3
knpp = 5
knpp2 = 15

lat_args.layers=[
    ('specific', (input_channel, knpp, kernel_size, input_height*input_width)),
    ('specific', (knpp, kernel_size, knpp2, input_height*input_width)),
    ('linear', (knpp2*input_height, lat_args.predict_len, (1,2), 1, (1, lat_args.predict_len))),
]

In [40]:
# mnist_args = dotdict()
mnist_args.train = dotdict()
mnist_args.predict = dotdict()

mnist_args.name = 'mnist'
mnist_args.train.batch_size = 60
mnist_args.predict.batch_size = 100

mnist_args.train_epochs = 10
mnist_args.lr = 0.0001
mnist_args.criterion = 'CE'
mnist_args.use_gpu = True

input_channel = 1
input_height = 28
input_width = 28
kernel_size = 9
knpp = 10
knpp2 = 20
knpp3 = 40

# mnist_args.layers=[
#     ('specific', (input_channel, knpp, kernel_size, input_height*input_width, (2,2))),
#     ('specific', (knpp, knpp2, kernel_size, input_height*input_width/4, (2,2))),
#     ('specific', (knpp2, knpp3, kernel_size, input_height*input_width/16, (1,1))),
#     ('linear', (int(knpp3*input_height*input_width/16), 10, (1,2,3))),
#     ('softmax', (1))
# ]

mnist_args.layers=[
    ('cnn2d', (input_channel, knpp, (3,3), 2, 1)),
    ('cnn2d', (knpp, knpp2, (3,3), 2, 1)),
    ('cnn2d', (knpp2, knpp3, (3,3), 1, 1)),
    ('linear', (int(knpp3*input_height*input_width/16), 10, (1,2,3))),
    ('softmax', (1))
]

In [41]:
cifar10_args = dotdict()
cifar10_args.train = dotdict()
cifar10_args.predict = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train.batch_size = 60
cifar10_args.predict.batch_size = 100

cifar10_args.train_epochs = 10
cifar10_args.lr = 0.0001
cifar10_args.criterion = 'CE'
cifar10_args.use_gpu = True

input_channel = 3
input_height = 32
input_width = 32
kernel_size = 9
knpp = 21
knpp2 = 84
knpp3 = 164

cifar10_args.layers=[
    ('agnostic', (input_channel, knpp, kernel_size, input_height*input_width, (2,2))),
    ('agnostic', (knpp, knpp2, kernel_size, input_height*input_width/4, (2,2))),
    ('agnostic', (knpp2, knpp3, kernel_size, input_height*input_width/16, (2,2))),
    ('linear', (int(knpp3*input_height*input_width/64), 10, (1,2,3))),
    ('softmax', (1))
]

# cifar10_args.layers=[
#     ('large', (input_channel, knpp, (5,5), 9, (2,2))),
#     ('large', (knpp, knpp2, (5,5), 9, (2,2))),
#     ('large', (knpp2, knpp3, (5,5), 9, (2,2))),
#     ('linear', (int(knpp3*input_height*input_width/64), 10, (1,2,3))),
#     ('softmax', (1))
# ]

# cifar10_args.layers=[
#     ('cnn2d', (input_channel, knpp, (3,3), 2, 1)),
#     ('cnn2d', (knpp, knpp2, (3,3), 2, 1)),
#     ('cnn2d', (knpp2, knpp3, (3,3), 2, 1)),
#     ('linear', (int(knpp3*input_height*input_width/64), 10, (1,2,3))),
#     ('softmax', (1))
# ]

SyntaxError: invalid syntax (3411212584.py, line 20)

In [42]:
args = mnist_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data)
driver.train()

Use GPU: cuda:None
epoch: 0, train_loss: 1.674263527750969
epoch: 1, train_loss: 1.5204044941663741
epoch: 2, train_loss: 1.5022865289449692
epoch: 3, train_loss: 1.493240487098694
epoch: 4, train_loss: 1.4876034020185471
epoch: 5, train_loss: 1.483711611032486
epoch: 6, train_loss: 1.4808115310668946
epoch: 7, train_loss: 1.4785899686813355
epoch: 8, train_loss: 1.4768556007146836
epoch: 9, train_loss: 1.4754529345035552


In [43]:
driver.metric()

0.9846

In [32]:
driver.metric()

0.9796

In [33]:
sum(p.numel() for p in driver.model.parameters() if p.requires_grad)

479540

In [12]:
train_pred = driver.predict(driver.data_loader.train).to(float).round()
train_pred

tensor([[[[  2.,   1.,  20.,  ...,  28.,  56.,   1.]]],


        [[[  1.,   0.,  -1.,  ...,  54.,  97.,   0.]]],


        [[[  3.,   1.,  21.,  ...,  36.,  76.,   0.]]],


        ...,


        [[[ 16.,   5.,   4.,  ..., 119., 152.,   1.]]],


        [[[ 10.,   5.,   2.,  ...,  80., 178.,   4.]]],


        [[[ 23.,   6.,   8.,  ..., 115., 134.,   2.]]]], dtype=torch.float64)

In [13]:
pred = driver.predict().to(float)
pred

tensor([[[[  5.7886,   2.5833,   2.7232,  ...,  51.8400, 157.8637,   3.4961]]]],
       dtype=torch.float64)

In [14]:
data.mean()

Region  Event
AA      01        10.700
        02         4.230
        03         7.100
        04        24.525
        05         5.190
                  ...   
ZI      16        45.055
        17       261.165
        18        94.235
        19       199.690
        20         1.025
Length: 5200, dtype: float64